In [1]:
# ADVENT OF CODE
# https://adventofcode.com/2020/day/20
# PREPARE DATA

import math
import re
import numpy as np

with open('input.sql', 'r') as f:
    inputfile = f.read()    
    tiles = dict([(int(y[0].replace(':','').replace('Tile ','')) , y[1:]) for y in [x.split("\n") for x in inputfile.split('\n\n')]])    

In [2]:
# get possible borders
def getPossibleBorders(tile):
    borders = []
    l = len(tile)    
    tileT =  [''.join(x) for x in [*zip(*tile)]]
    borders += [tile[x] for x in [0,l-1]] # atas, bawah
    borders += [tileT[x] for x in [0,l-1]] # atas, bawah
    borders += [x[::-1] for x in borders] # flip
    
    return set(borders)

def findAdjacent(targetkey, target, tile_borders):
    result = set()    
    for key in tile_borders:
        for border in tile_borders[key]['self']:
            if target ==  border and key != targetkey: 
                result.add(key)
    
    return result

def findTileBorders(tiles):  
    tile_borders = {}
    for key in tiles:    
        tile_borders[key] = {'self': [], 'adjacents': set() }
        tile_borders[key]['self'] = getPossibleBorders(tiles[key])

    result = []
    for key in tile_borders:
        adjacents = set()
        for border in tile_borders[key]['self']:
            adjacents = adjacents | findAdjacent(key, border, tile_borders)
            tile_borders[key]['adjacents'] = adjacents
            
    return tile_borders
            
def findCorners(tile_borders):       
    corners = [ID for ID in tile_borders if len(tile_borders[ID]['adjacents']) == 2]
    return corners, math.prod(corners)

def tilerot90(tile):
    return [''.join(x) for x in [*zip(*tile[::-1])]]

def addRight(left, right):
    if len(left) == 0:
        return right
    else:
        return [''.join(x) for x in list(zip(left, right))]

def checkRight(left, right):
    return ''.join([row[-1] for row in left]) == ''.join([row[0] for row in right])

def checkBottom(top, bottom):
    return top[-1] == bottom[0]

def checkTopRight(left, right, top):       
    a = ''.join([row[0] for row in right])
    b = ''.join([row[-1] for row in left])
    c = top[-1]
    d = right[0]

    return a == b and c==d

def matchTile(toadd, left=None, top = None):
    tileA = left
    tileB = toadd
    tileB2 = toadd[::-1]
    
    if top is not None:
        tileC = top
    
    if not left is None:   
        for i in range(4):                     
            check = checkRight(tileA, tileB) if top is None else checkTopRight(tileA, tileB, tileC)
            if (check):
                return tileB
            else:
                tileB = tilerot90(tileB)    

            check2 = checkRight(tileA, tileB2) if top is None else checkTopRight(tileA, tileB2, tileC)   
            if (check2):            
                return tileB2           
            else:
                tileB2 = tilerot90(tileB2) 
    else:
        if not top is None:
            for i in range(4):   
                check = checkBottom(tileC, tileB)
                if (check):
                    return tileB
                else:
                    tileB = tilerot90(tileB)    
        
                check2 = checkBottom(tileC, tileB2) 
                if (check2):            
                    return tileB2           
                else:
                    tileB2 = tilerot90(tileB2)      

def getMergedTiles(tiles, tile_borders):
    corners, corners_count = findCorners(tile_borders)    
    size = int(math.sqrt(len(tiles)))
    merged = [tiles[corners[0]]]
    address = [corners[0]]
    x = 1 
    while True:       
        for toAdd in [x for x in list(tile_borders[address[x - 1 if x % size != 0 else x-size]]['adjacents']) if x not in address]:                  
            if toAdd not in address:    
                if x % size == 0:                    
                    result = matchTile(top = merged[x - size], toadd = tiles[toAdd])  
                    if result is None:                    
                        result = matchTile(top = merged[x - size][::-1], toadd = tiles[toAdd])  
                        merged = [x[::-1] for x in merged]
                elif x <= size:                       
                    result = matchTile(left = merged[-1], toadd = tiles[toAdd])
                else:                
                    result = matchTile(left = merged[-1], toadd = tiles[toAdd], top = merged[x - size])

            if result is not None:
                address.append(toAdd)
                merged.append(result)     
                x += 1       
                break     

        if x == size**2:
            break
        
    return address, merged

def removeBorder(tile):
    return [x[1:-1] for x in tile][1:-1]

def finalImage(merged):
    final_image = row = []     
    size = int(math.sqrt(len(merged)))

    for index, tile in enumerate(merged):       
        if (index) % size == 0:        
            row = removeBorder(tile)    
        else:
            row = addRight(row, removeBorder(tile))        

        if (index+1) % size == 0:
            final_image.extend(row)
            
    return final_image

def findMoster(image):    
    distance = str(len(image[0]) - 20 + 1)
    monster = "(?=.{18}#.{"+distance+"}#.{4}##.{4}##.{4}###.{"+distance+"}#..#..#..#..#..#...)"
    curImage = image
    i = j = 0
    while i < 4:
        while j < 2:        
            OneDimage = ''.join(curImage);
            found = re.findall(monster, OneDimage)
            if len(found) > 0:            
                return OneDimage.count('#') - monster.count('#') * len(found)
            
            curImage = tilerot90(list(reversed(curImage)))
            j += 1
        i += 1 
        curImage = tilerot90(curImage)
    
    return 0
        

## PROBLEM 1

In [3]:
tile_borders = findTileBorders(tiles)
findCorners(tile_borders)

([1753, 2843, 3083, 1489], 22878471088273)

## PROBLEM 2

In [4]:
address, merged = getMergedTiles(tiles, tile_borders)
final_image = finalImage(merged)
findMoster(final_image)

1680